# Observed Air Quality (PurpleAir)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [52]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import time # For sleep in for loop

# Analysis

import numpy as np
import pandas as pd

## Definitions

In [16]:
spike_threshold = 28 # Micgrograms per meter cubed

In [17]:
# This is my personal API key... Please use responsibly!

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


### Sensor Indices and Start Times

In [53]:
# Sensor Indices (from City of Minneapolis)

sensor_info = pd.read_excel('PA IDs and indexes.xlsx') # Load as DataFrame

sensor_ids = sensor_info['Sensor Index'].dropna().astype(int) # This should be an iterable of the sensor ids as integers

In [54]:
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    # print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

In [55]:
# Get start Times

sensor_string = '%2C'.join(sensor_ids.astype(str))

query = 'fields=date_created&show_only=' + sensor_string

response = getSensorsData(query, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

In [56]:
sensors_df.head()

,sensor_index,date_created
0,142718,1642013869
1,142720,1642013875
2,142726,1642013897
3,142724,1642013889
4,142730,1642013916


### Summary Statistics Functions

In [22]:
%run Summary_Functions.py

print('Stat Names:\n\n', summary_stats_names, '\n')
print('Stat Types:\n\n',summary_stats_dtypes, '\n')

print('Function Names:\n\n', summary_stats_functions)

Stat Names:

 ['n_observations', 'pm25_fullDay_mean', 'pm25_fullDay_min', 'pm25_fullDay_minTime', 'pm25_fullDay_max', 'pm25_fullDay_maxTime', 'pm25_fullDay_std', 'pm25_fullDay_minutesAbove12ug', 'pm25_morningRush_mean', 'pm25_morningRush_min', 'pm25_morningRush_minTime', 'pm25_morningRush_max', 'pm25_morningRush_maxTime', 'pm25_morningRush_std', 'pm25_eveningRush_mean', 'pm25_eveningRush_min', 'pm25_eveningRush_minTime', 'pm25_eveningRush_max', 'pm25_eveningRush_maxTime', 'pm25_eveningRush_std', 'pm25_daytimeAmbient_mean', 'pm25_daytimeAmbient_min', 'pm25_daytimeAmbient_minTime', 'pm25_daytimeAmbient_max', 'pm25_daytimeAmbient_maxTime', 'pm25_daytimeAmbient_std', 'pm25_nighttimeAmbient_mean', 'pm25_nighttimeAmbient_min', 'pm25_nighttimeAmbient_minTime', 'pm25_nighttimeAmbient_max', 'pm25_nighttimeAmbient_maxTime', 'pm25_nighttimeAmbient_std'] 

Stat Types:

 [<class 'int'>, <class 'float'>, <class 'float'>, <class 'str'>, <class 'float'>, <class 'str'>, <class 'float'>, <class 'int'>, 

### Functions

In [23]:
# QAQC

def qaqc(df):
    '''This function wil perform some basic QAQC
    '''
    
    clean_df = df.copy()
    
    # Convert timestamp to datetime
    
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'], unit='s')
    
    # Remove obvious error values
    
    clean_df = clean_df[clean_df.pm25 < 1000] 
    
    # Remove NaNs
    
    clean_df = clean_df.dropna()
    
    return clean_df

# Remove and record Spikes

def get_spikes(df, spike_threshold):
    '''This function removes spikes from a dataframe 
    and returns both the new dataframe
    and a separate spike dataframe
    '''
    
    df_w_spikes = df.copy()
    
    condition = (df.pm25 > spike_threshold)
    
    df_w_spikes['is_spike'] = condition
    
    spikes = df_w_spikes[condition].copy()
    
    return df_w_spikes, spikes

# Get Summary Stats

def get_summary_stats(df):
    ''' This is the main function. It will run all of our functions that get summary stats
    and return as a list.
    '''
    
    stats = []
    
    # Run the functions
    
    for f in summary_stats_functions:
        stats += f(df)
    
    return stats

### Set Up Parameters for Query

In [24]:
### Query Strings

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=10'

# Environmental fields

env_fields = ['humidity', 'temperature', 'pressure', 'pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# My Header

my_headers = {'X-API-Key': api}

## The Loop

In [63]:
## Iterables

# Dates

# first_date = pd.to_datetime(sensors_df.date_created, unit = 's').min()
# ^ This is just untrue...


first_date = dt.datetime(2022, 6, 15) # June 15th, 2022?

datelist = pd.date_range(start = first_date, 
                         end = dt.datetime.today(),
                        normalize = True)

print('Last Run on ', dt.datetime.today())

Last Run on  2023-05-01 00:24:25.981869


In [64]:
## Initialize Storage

# If this script has already been run, please replace the dataframes with their respective .csvs

has_run = True

if has_run:
    
    data_path = os.path.join('..','..','..', 'data')
    
    daily_summary_df = pd.read_csv(os.path.join(data_path, 'daily_summaries.csv'))
    
    daily_summary_no_spikes_df = pd.read_csv(os.path.join(data_path, 'daily_summaries_no_spikes.csv'))
    
    all_spikes_df = pd.read_csv(os.path.join(data_path, 'all_spikes.csv'))
    all_spikes_df['timestamp'] = pd.to_datetime(all_spikes_df.timestamp)
    
    no_data = pd.read_csv(os.path.join(data_path, 'no_data_sensors.csv'))
                                   
    
else:
    
    # Daily Summary

    cols = ['sensor_index', 'date'] + summary_stats_names

    datatypes = [int, str] + summary_stats_dtypes

    dtypes = np.dtype(list(zip(cols, datatypes)))

    daily_summary_df = pd.DataFrame(np.empty(0, dtype = dtypes))

    # Daily Summary (No Spikes)

    cols = ['sensor_index', 'date'] + summary_stats_names

    datatypes = [int, str] + summary_stats_dtypes

    dtypes = np.dtype(list(zip(cols, datatypes)))

    daily_summary_no_spikes_df = pd.DataFrame(np.empty(0, dtype = dtypes))

    # Spikes

    all_spikes_df = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                      ('timestamp', pd._libs.tslibs.timestamps.Timestamp),
                                                      ('pm25', float)]
                                     )
                            )

    # No Data for sensor

    no_data = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                ('date', str)
                                               ]))

In [65]:
# Iterate through the days

for i in range(len(datelist)-1): 
    
    # Set up Timestamp for query    
    
    start_timestamp = int(datelist[i].timestamp())
    end_timestamp = int(datelist[i+1].timestamp())
    
    time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)
    
    # Select Sensors that had been created before this date
    
    select_sensors = sensors_df[sensors_df.date_created <= start_timestamp]
    
    sensor_ids = select_sensors.sensor_index
    
    # Iterate through the Sensors
    
    for sensor_id in sensor_ids:
        
        # For skipping to last spot
        
        is_done = (daily_summary_df.sensor_index == int(sensor_id)) & (daily_summary_df.date == str(datelist[i].date()))
        is_no_data = (no_data.sensor_index == int(sensor_id)) & (no_data.date == str(datelist[i].date()))
        
        # If either of these has a true, it has been parsed
        # True is not in both, then we should process
        if (True not in is_done.values) & (True not in is_no_data.values):
            
            ### Actual Loop

            time.sleep(3)

            # Base URL
            base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

            # Put it all together
            query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

            response = requests.get(query_url, headers=my_headers)

            if response.status_code == 200:

                # Read response as CSV data
                csv_data = response.content.decode('utf-8')

                if csv_data.count('\n') == 1: # There is only one line (empty data)
                    # print(f"No data for sensor {sensor_id} on {datelist[i]}")

                    no_data.loc[len(no_data.index)] = [sensor_id, str(datelist[i].date())]

                else:
                    # Parse CSV data into pandas DataFrame
                    df_individual_sensor = pd.read_csv(io.StringIO(csv_data),
                                                       header=0
                                                      )[['time_stamp', 'humidity', 'temperature', 
                                                         'pressure', 'pm2.5_cf_1']]

                    df_individual_sensor.columns = ['timestamp','humidity', 'temperature', 
                                                    'pressure', 'pm25']

                    # Perform QAQC

                    clean = qaqc(df_individual_sensor)

                    # Remove Spikes & Concatenate to main storage of spikes

                    clean_w_spikes, spikes = get_spikes(clean, spike_threshold)

                    spikes['sensor_index'] = int(sensor_id)

                    all_spikes_df = pd.concat([all_spikes_df, 
                                               spikes[['sensor_index',
                                                        'timestamp',
                                                        'pm25']]
                                              ],
                                               ignore_index=True)

                    # Get Stats (With Spikes)

                    sum_stats = get_summary_stats(clean_w_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_df.loc[len(daily_summary_df.index)] = row

                    # Get Stats (Without Spikes)

                    no_spikes = clean_w_spikes[clean_w_spikes.is_spike == False]

                    sum_stats = get_summary_stats(no_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_no_spikes_df.loc[len(daily_summary_no_spikes_df.index)] = row

            else:
                print(f"Error fetching data for sensor {sensor_id}: {response.status_code} on {datelist[i].date()}")
    
    if not has_run:
    
        # Save it!?! After a day is processed, yes!

        daily_summary_df.to_csv('daily_summaries.csv', index = False)

        daily_summary_no_spikes_df.to_csv('daily_summaries_no_spikes.csv', index = False)
    
    # For testing a week
    
    # if i == 7:
    #     break

In [66]:
daily_summary_df.tail(10)

# Drop Duplicates
# daily_summary_df = daily_summary_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,pm25_fullDay_maxTime,pm25_fullDay_std,pm25_fullDay_minutesAbove12ug,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
12028,157787,2023-04-30,144,0.979063,0.000,11:30:00,15.825,03:40:00,2.665879,50,...,12:00:00,0.169,14:30:00,0.044305,2.558947,1.192,00:10:00,12.448,03:00:00,2.522875
12029,157785,2023-04-30,98,0.869245,0.000,09:30:00,3.165,02:40:00,0.874179,0,...,13:20:00,0.313,14:30:00,0.064133,2.220263,1.322,00:40:00,3.165,02:40:00,0.538197
12030,157837,2023-04-30,144,0.650347,0.000,11:40:00,4.172,03:40:00,0.814672,0,...,12:10:00,0.466,14:30:00,0.105313,1.812316,1.337,01:20:00,2.493,03:00:00,0.347042
12031,157845,2023-04-30,144,1.484000,0.000,10:10:00,12.658,00:30:00,2.152730,10,...,12:10:00,11.989,13:00:00,3.747244,3.991211,1.922,00:00:00,12.658,00:30:00,2.356211
12032,157861,2023-04-30,144,0.737812,0.000,11:40:00,3.170,03:10:00,0.789879,0,...,12:00:00,0.284,14:30:00,0.077754,2.114684,1.504,00:40:00,2.805,02:50:00,0.390935
12033,157871,2023-04-30,144,28.055299,0.000,11:50:00,495.569,00:50:00,88.126005,190,...,12:50:00,0.235,14:30:00,0.071893,204.924842,1.796,00:10:00,495.569,00:50:00,153.476516
12034,157877,2023-04-30,144,0.702354,0.000,09:00:00,3.899,00:20:00,0.881890,0,...,13:10:00,0.262,14:40:00,0.079066,2.489947,1.856,01:30:00,3.899,00:20:00,0.464620
12035,157935,2023-04-30,144,0.384528,0.000,09:50:00,2.069,03:10:00,0.516696,0,...,12:00:00,0.102,14:20:00,0.035080,1.360053,0.945,00:20:00,1.992,02:50:00,0.339937
12036,166459,2023-04-30,144,1.724944,0.513,09:30:00,3.750,03:00:00,0.794999,0,...,12:30:00,1.600,13:40:00,0.180561,3.190368,2.287,00:00:00,3.750,03:00:00,0.412136
12037,168327,2023-04-30,144,0.663632,0.000,10:40:00,2.424,02:50:00,0.676697,0,...,12:00:00,0.305,14:50:00,0.091038,1.879474,1.393,00:30:00,2.424,02:50:00,0.318709


In [67]:
daily_summary_no_spikes_df.tail(10)

# To Remove Duplicates
# daily_summary_no_spikes_df = daily_summary_no_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,pm25_fullDay_maxTime,pm25_fullDay_std,pm25_fullDay_minutesAbove12ug,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
12028,157787,2023-04-30,144,0.979063,0.000,11:30:00,15.825,03:40:00,2.665879,50,...,12:00:00,0.169,14:30:00,0.044305,2.558947,1.192,00:10:00,12.448,03:00:00,2.522875
12029,157785,2023-04-30,98,0.869245,0.000,09:30:00,3.165,02:40:00,0.874179,0,...,13:20:00,0.313,14:30:00,0.064133,2.220263,1.322,00:40:00,3.165,02:40:00,0.538197
12030,157837,2023-04-30,144,0.650347,0.000,11:40:00,4.172,03:40:00,0.814672,0,...,12:10:00,0.466,14:30:00,0.105313,1.812316,1.337,01:20:00,2.493,03:00:00,0.347042
12031,157845,2023-04-30,144,1.484000,0.000,10:10:00,12.658,00:30:00,2.152730,10,...,12:10:00,11.989,13:00:00,3.747244,3.991211,1.922,00:00:00,12.658,00:30:00,2.356211
12032,157861,2023-04-30,144,0.737812,0.000,11:40:00,3.170,03:10:00,0.789879,0,...,12:00:00,0.284,14:30:00,0.077754,2.114684,1.504,00:40:00,2.805,02:50:00,0.390935
12033,157871,2023-04-30,127,0.968323,0.000,11:50:00,13.561,03:20:00,1.970171,20,...,12:50:00,0.235,14:30:00,0.071893,5.458333,1.796,00:10:00,12.498,03:00:00,6.098195
12034,157877,2023-04-30,144,0.702354,0.000,09:00:00,3.899,00:20:00,0.881890,0,...,13:10:00,0.262,14:40:00,0.079066,2.489947,1.856,01:30:00,3.899,00:20:00,0.464620
12035,157935,2023-04-30,144,0.384528,0.000,09:50:00,2.069,03:10:00,0.516696,0,...,12:00:00,0.102,14:20:00,0.035080,1.360053,0.945,00:20:00,1.992,02:50:00,0.339937
12036,166459,2023-04-30,144,1.724944,0.513,09:30:00,3.750,03:00:00,0.794999,0,...,12:30:00,1.600,13:40:00,0.180561,3.190368,2.287,00:00:00,3.750,03:00:00,0.412136
12037,168327,2023-04-30,144,0.663632,0.000,10:40:00,2.424,02:50:00,0.676697,0,...,12:00:00,0.305,14:50:00,0.091038,1.879474,1.393,00:30:00,2.424,02:50:00,0.318709


In [68]:
all_spikes_df

# To Remove Duplicates
# all_spikes_df = all_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,timestamp,pm25
0,143226,2022-06-17 22:10:00,58.910
1,143226,2022-06-17 22:20:00,35.495
2,143226,2022-06-17 22:00:00,319.660
3,142720,2022-06-18 03:20:00,33.595
4,142720,2022-06-18 03:30:00,36.026
...,...,...,...
175256,157871,2023-04-30 02:20:00,64.306
175257,157871,2023-04-30 02:40:00,122.301
175258,157871,2023-04-30 01:50:00,396.923
175259,157871,2023-04-30 03:10:00,39.789


In [69]:
# no_data['date'] = pd.to_datetime(no_data.date).dt.date.astype(str) # If need to correct the dates
no_data
# To Remove Duplicates
# no_data = no_data.drop_duplicates(ignore_index = True).copy()

,sensor_index,date
0,142718,2022-06-15
1,142720,2022-06-15
2,142726,2022-06-15
3,142730,2022-06-15
4,142728,2022-06-15
...,...,...
7974,145262,2023-04-30
7975,145502,2023-04-30
7976,145504,2023-04-30
7977,145610,2023-04-30


In [70]:
daily_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12038 entries, 0 to 12037
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sensor_index                   12038 non-null  int64  
 1   date                           12038 non-null  object 
 2   n_observations                 12038 non-null  int64  
 3   pm25_fullDay_mean              11932 non-null  float64
 4   pm25_fullDay_min               11932 non-null  float64
 5   pm25_fullDay_minTime           11932 non-null  object 
 6   pm25_fullDay_max               11932 non-null  float64
 7   pm25_fullDay_maxTime           11932 non-null  object 
 8   pm25_fullDay_std               11913 non-null  float64
 9   pm25_fullDay_minutesAbove12ug  12038 non-null  int64  
 10  pm25_morningRush_mean          11617 non-null  float64
 11  pm25_morningRush_min           11617 non-null  float64
 12  pm25_morningRush_minTime       11617 non-null 

In [71]:
# # Save a test dataframe

# clean_w_spikes.to_csv('example_df.csv', index = False)

# Save it!?!

savepath = os.path.join('..', '..','..', 'data')

daily_summary_df.to_csv(os.path.join(savepath, 'daily_summaries.csv'), index = False)

daily_summary_no_spikes_df.to_csv(os.path.join(savepath, 'daily_summaries_no_spikes.csv'), index = False)

all_spikes_df.to_csv(os.path.join(savepath, 'all_spikes.csv'), index = False)

no_data.to_csv(os.path.join(savepath, 'no_data_sensors.csv'), index = False)